## Machine Learning 과제 1: 야구 선수 연봉 예측

* 문제1: 최동원 선수와 다른 투수들의 1983~1988 데이터 / 2015~2020 프로야구 투수 데이터 확보
  - 선수 기록: https://www.koreabaseball.com/Record/Player/PitcherBasic/BasicOld.aspx
  - 선수 연봉: http://www.statiz.co.kr/salary.php

In [2]:
# requirements

from selenium import webdriver
from selenium.webdriver.common.by import By
import time
from bs4 import BeautifulSoup
import pandas as pd
import numpy as np
import re
import math

In [3]:
# opening a tab with selenium / 선수기록 먼저 확보

URL = "https://www.koreabaseball.com/Record/Player/PitcherBasic/BasicOld.aspx"
driver = webdriver.Chrome("../driver/chromedriver.exe")
driver.get(URL) # KBO 선수 기록실 사이트

## TEST

In [4]:
# 연도 선택 (예시로 1983 선택 >> 전체 완료되면 변경해야 됨)

'''driver.find_element_by_css_selector('#cphContents_cphContents_cphContents_ddlSeason_ddlSeason > option:nth-child(2)').click()'''

# 1983: '#cphContents_cphContents_cphContents_ddlSeason_ddlSeason > option:nth-child(2)'

# 1988: '#cphContents_cphContents_cphContents_ddlSeason_ddlSeason > option:nth-child(7)'

"driver.find_element_by_css_selector('#cphContents_cphContents_cphContents_ddlSeason_ddlSeason > option:nth-child(2)').click()"

In [5]:
# 페이지 html 읽기 & 첫번째 선수 record 가져오기
html = driver.page_source
dom = BeautifulSoup(html, 'html.parser')

record_detail = dom.select('#cphContents_cphContents_cphContents_udpContent > div.record_result > table > tbody > tr')

a = record_detail[0].text.split('\n')
len(record_detail)
a[1:-1]

['1',
 '페디',
 'NC',
 '1.74',
 '17',
 '0',
 '0',
 '14',
 '2',
 '0',
 '0',
 '0.875',
 '409',
 '103 1/3',
 '77',
 '6',
 '24',
 '2',
 '121',
 '23',
 '20']

## 83~88년도 선수 데이터 획득 및 데이터프레임화

In [6]:
team = driver.find_element_by_css_selector('#cphContents_cphContents_cphContents_ddlTeam_ddlTeam')
len(team.text.split('\n'))

12

In [7]:
#cphContents_cphContents_cphContents_ddlTeam_ddlTeam > option:nth-child(2)

#cphContents_cphContents_cphContents_ddlTeam_ddlTeam > option:nth-child(7)

In [8]:
# 1983~1988 년도 선수 데이터 획득

info8388 = []

# 년도 데이터 추가를 위한 보조 변수
a = 1982
b = 0

for n in range(2, 8):
    a += 1
    driver.find_element_by_css_selector(f'#cphContents_cphContents_cphContents_ddlSeason_ddlSeason > option:nth-child({n})').click()

    time.sleep(0.7)

    team = driver.find_element_by_css_selector('#cphContents_cphContents_cphContents_ddlTeam_ddlTeam')

    for y in range(2, len(team.text.split('\n'))+1):
        driver.find_element_by_css_selector(f'#cphContents_cphContents_cphContents_ddlTeam_ddlTeam > option:nth-child({y})').click()

        time.sleep(0.7)

        html = driver.page_source
        dom = BeautifulSoup(html, 'html.parser')

        record_detail = dom.select('#cphContents_cphContents_cphContents_udpContent > div.record_result > table > tbody > tr')

        for x in range(len(record_detail)):
            info8388.append(record_detail[x].text.split('\n')[1:-1])
            info8388[b].append(a)
            b += 1

    

In [10]:
len(info8388)

452

In [11]:
info8388

[['1',
  '최동원',
  '롯데  ',
  '2.89',
  '38',
  '16',
  '1',
  '9',
  '16',
  '4',
  '0',
  '0.360',
  '863',
  '208 2/3',
  '202',
  '17',
  '51',
  '8',
  '148',
  '89',
  '67',
  1983],
 ['2',
  '배경환',
  '롯데  ',
  '3.18',
  '37',
  '4',
  '2',
  '7',
  '4',
  '0',
  '0',
  '0.636',
  '637',
  '150',
  '159',
  '9',
  '32',
  '5',
  '62',
  '72',
  '53',
  1983],
 ['3',
  '천창호',
  '롯데  ',
  '3.42',
  '27',
  '0',
  '0',
  '7',
  '3',
  '0',
  '0',
  '0.700',
  '349',
  '81 2/3',
  '91',
  '12',
  '12',
  '6',
  '38',
  '39',
  '31',
  1983],
 ['4',
  '김문희',
  '롯데  ',
  '3.48',
  '34',
  '5',
  '0',
  '8',
  '10',
  '0',
  '0',
  '0.444',
  '649',
  '155 1/3',
  '154',
  '12',
  '53',
  '3',
  '46',
  '70',
  '60',
  1983],
 ['5',
  '노상수',
  '롯데  ',
  '4.14',
  '42',
  '4',
  '1',
  '8',
  '11',
  '3',
  '0',
  '0.421',
  '733',
  '169 2/3',
  '179',
  '17',
  '52',
  '12',
  '69',
  '94',
  '78',
  1983],
 ['6',
  '이진우',
  '롯데  ',
  '4.94',
  '27',
  '0',
  '0',
  '4',
  '9',
  '0',
  

In [12]:
# 83~88년도 선수 정보 데이터 프레임 생성
df_8388 = pd.DataFrame({
    '순위':[],
    '선수명':[],
    '팀명':[],
    'ERA':[],
    'G':[],
    'CG':[],
    'SHO':[],
    'W':[],
    'L':[],
    'SV':[],
    'HLD':[],
    'WPCT':[],
    'TBF':[],
    'IP':[],
    'H':[],
    'HR':[],
    'BB':[],
    'HBP':[],
    'SO':[],
    'R':[],
    'ER':[],
    '년도':[]
    })

In [13]:
# 데이터프레임에 획득한 데이터 삽입

for n in range(len(info8388)):
    df_8388.loc[n] = info8388[n]

In [14]:
# 데이터 프레임 확인

df_8388.tail()

,순위,선수명,팀명,ERA,G,CG,SHO,W,L,SV,...,TBF,IP,H,HR,BB,HBP,SO,R,ER,년도
447,8,김강익,OB,5.28,6,0,0,0,1,0,...,63,15 1/3,15,4,5,0,3,9,9,1988
448,9,박상열,OB,5.59,13,0,0,0,3,0,...,216,46 2/3,53,9,23,4,12,34,29,1988
449,10,황태환,OB,6.52,23,1,0,0,2,0,...,221,49 2/3,58,7,21,1,11,38,36,1988
450,11,하창우,OB,6.61,5,1,0,0,1,0,...,70,16 1/3,22,1,3,1,2,13,12,1988
451,12,박노준,OB,18.00,1,0,0,0,0,0,...,17,3,5,0,1,1,1,6,6,1988


## 15~20년도 선수 데이터 획득 및 데이터 프레임화

In [15]:
# 15년도 선택 cphContents_cphContents_cphContents_ddlSeason_ddlSeason > option:nth-child(34)

# 20년도 선택 cphContents_cphContents_cphContents_ddlSeason_ddlSeason > option:nth-child(39)

In [16]:
driver.find_element_by_css_selector(f'#cphContents_cphContents_cphContents_ddlSeason_ddlSeason > option:nth-child({34})').click()

time.sleep(0.3)

html = driver.page_source
dom = BeautifulSoup(html, 'html.parser')

record_detail = dom.select('#cphContents_cphContents_cphContents_udpContent > div.record_result > table > tbody > tr')

In [17]:
record_detail[0]

<tr>
<td>1</td>
<td><a href="/Record/Retire/Pitcher.aspx?playerId=79544">오장훈</a></td>
<td>두산</td>
<td class="asc" data-id="ERA_RT">0.00</td>
<td data-id="GAME_CN">1</td>
<td data-id="W_CN">0</td>
<td data-id="L_CN">0</td>
<td data-id="SV_CN">0</td>
<td data-id="HOLD_CN">0</td>
<td data-id="WRA_RT">-</td>
<td data-id="INN2_CN">1</td>
<td data-id="HIT_CN">1</td>
<td data-id="HR_CN">0</td>
<td data-id="BB_CN">1</td>
<td data-id="HP_CN">0</td>
<td data-id="KK_CN">2</td>
<td data-id="R_CN">0</td>
<td data-id="ER_CN">0</td>
<td data-id="WHIP_RT">2.00</td>
</tr>

In [19]:
# 2015~2020 년도 선수 데이터 획득 (1)

info1520 = []

for n in range(34, 40):
    driver.find_element_by_css_selector(f'#cphContents_cphContents_cphContents_ddlSeason_ddlSeason > option:nth-child({n})').click()

    time.sleep(1)

    team = driver.find_element_by_css_selector('#cphContents_cphContents_cphContents_ddlTeam_ddlTeam')

    for y in range(2, len(team.text.split('\n'))+1):
        driver.find_element_by_css_selector(f'#cphContents_cphContents_cphContents_ddlTeam_ddlTeam > option:nth-child({y})').click()

        time.sleep(1)

        html = driver.page_source
        dom = BeautifulSoup(html, 'html.parser')

        record_detail = dom.select('#cphContents_cphContents_cphContents_udpContent > div.record_result > table > tbody > tr')

        for x in range(len(record_detail)):
            info1520.append(record_detail[x].text.split('\n')[1:-1])

        time.sleep(0.5)
        
        try: # 2번째 페이지 존재시 추가 데이터 획득
            driver.find_element_by_css_selector('#cphContents_cphContents_cphContents_ucPager_btnNo2').click()

            time.sleep(1)

            html = driver.page_source
            dom = BeautifulSoup(html, 'html.parser')

            record_detail = dom.select('#cphContents_cphContents_cphContents_udpContent > div.record_result > table > tbody > tr')

            for x in range(len(record_detail)):
                info1520.append(record_detail[x].text.split('\n')[1:-1])
            
            driver.find_element_by_css_selector('#cphContents_cphContents_cphContents_ucPager_btnNo1').click()

            time.sleep(2)
            
        except:
            continue

driver.find_element_by_css_selector('#cphContents_cphContents_cphContents_udpContent > div.row > div.more_record > a.next').click()

In [20]:
# 2015~2020 년도 선수 데이터 획득 (2) >> 기록보기로 인한 추가 데이터

info1520_tmp = []

# 년도 데이터 추가를 위한 보조 변수
a = 2014
b = 0

for n in range(34, 40):
    a += 1

    driver.find_element_by_css_selector(f'#cphContents_cphContents_cphContents_ddlSeason_ddlSeason > option:nth-child({n})').click()

    time.sleep(1)

    team = driver.find_element_by_css_selector('#cphContents_cphContents_cphContents_ddlTeam_ddlTeam')

    for y in range(2, len(team.text.split('\n'))+1):
        driver.find_element_by_css_selector(f'#cphContents_cphContents_cphContents_ddlTeam_ddlTeam > option:nth-child({y})').click()

        time.sleep(1)

        html = driver.page_source
        dom = BeautifulSoup(html, 'html.parser')

        record_detail = dom.select('#cphContents_cphContents_cphContents_udpContent > div.record_result > table > tbody > tr')

        for x in range(len(record_detail)):
            info1520_tmp.append(record_detail[x].text.split('\n')[1:-1])
            info1520_tmp[b].append(a)
            b += 1

        try: # 2번째 페이지 존재시 추가 데이터 획득
            driver.find_element_by_css_selector('#cphContents_cphContents_cphContents_ucPager_btnNo2').click()

            time.sleep(1)

            html = driver.page_source
            dom = BeautifulSoup(html, 'html.parser')

            record_detail = dom.select('#cphContents_cphContents_cphContents_udpContent > div.record_result > table > tbody > tr')

            for x in range(len(record_detail)):
                info1520_tmp.append(record_detail[x].text.split('\n')[1:-1])
                info1520_tmp[b].append(a)
                b += 1
            driver.find_element_by_css_selector('#cphContents_cphContents_cphContents_ucPager_btnNo1').click()

            time.sleep(2)
            
        except:
            continue

In [21]:
# 위 두 데이터셋 합치기

for n in range(len(info1520)):
    for x in range(4, 19):
        info1520[n].append(info1520_tmp[n][x])

In [22]:
# 15~20년도 선수 정보 데이터 프레임 생성

df_1520 = pd.DataFrame({
    '순위':[],
    '선수명':[],
    '팀명':[],
    'ERA':[],
    'G':[],
    'W':[],
    'L':[],
    'SV':[],
    'HLD':[],
    'WPCT':[],
    'IP':[],
    'H':[],
    'HR':[],
    'BB':[],
    'HBP':[],
    'SO':[],
    'R':[],
    'ER':[],
    'WHIP':[],
    'CG':[],
    'SHO':[],
    'QS':[],
    'BSV':[],
    'TBF':[],
    'NP':[],
    'AVG':[],
    '2B':[],
    '3B':[],
    'SAC':[],
    'SF':[],
    'IBB':[],
    'WP':[],
    'BK':[],
    '년도':[]
    })

In [23]:
# 데이터프레임에 획득한 데이터 삽입

for n in range(len(info1520)):
    df_1520.loc[n] = info1520[n]

In [24]:
df_1520.tail()

,순위,선수명,팀명,ERA,G,W,L,SV,HLD,WPCT,...,NP,AVG,2B,3B,SAC,SF,IBB,WP,BK,년도
1528,25,김택형,SK,8.61,31,1,3,0,3,0.250,...,439,0.280,2,1,1,0,1,6,0,2020
1529,26,양선률,SK,9.00,1,0,0,0,0,-,...,23,0.333,0,0,0,0,0,0,0,2020
1530,27,신재웅,SK,9.31,24,2,0,0,0,1.000,...,378,0.333,3,2,0,1,0,0,0,2020
1531,28,김찬호,SK,27.00,3,0,0,0,0,-,...,57,0.500,0,0,0,0,0,0,0,2020
1532,29,윤강민,SK,67.50,1,0,0,0,0,-,...,23,0.667,1,0,0,0,0,0,0,2020


In [25]:
df_8388.columns

Index(['순위', '선수명', '팀명', 'ERA', 'G', 'CG', 'SHO', 'W', 'L', 'SV', 'HLD',
       'WPCT', 'TBF', 'IP', 'H', 'HR', 'BB', 'HBP', 'SO', 'R', 'ER', '년도'],
      dtype='object')

In [26]:
df_1520.columns

Index(['순위', '선수명', '팀명', 'ERA', 'G', 'W', 'L', 'SV', 'HLD', 'WPCT', 'IP', 'H',
       'HR', 'BB', 'HBP', 'SO', 'R', 'ER', 'WHIP', 'CG', 'SHO', 'QS', 'BSV',
       'TBF', 'NP', 'AVG', '2B', '3B', 'SAC', 'SF', 'IBB', 'WP', 'BK', '년도'],
      dtype='object')

In [27]:
# 15~20년도 선수 데이터프레임에서 83~88 년도에 존재하지 않는 컬럼 확인

tmp = [x for x in list(df_1520.columns) if x not in list(df_8388.columns)]
tmp

['WHIP', 'QS', 'BSV', 'NP', 'AVG', '2B', '3B', 'SAC', 'SF', 'IBB', 'WP', 'BK']

In [28]:
# 존재하지 않는 컬럼 제거

df_1520 = df_1520.drop(tmp, axis=1)
df_1520.head()

,순위,선수명,팀명,ERA,G,W,L,SV,HLD,WPCT,...,HR,BB,HBP,SO,R,ER,CG,SHO,TBF,년도
0,1,김정인,넥센,0.00,1,0,0,0,0,-,...,0,0,0,0,0,0,0,0,3,2015
1,2,양훈,넥센,1.41,16,2,1,0,0,0.667,...,1,14,0,15,9,6,0,0,153,2015
2,3,조상우,넥센,3.09,70,8,5,5,19,0.615,...,4,41,6,89,36,32,0,0,382,2015
3,4,밴헤켄,넥센,3.62,32,15,8,0,0,0.652,...,14,67,1,193,92,79,0,0,825,2015
4,5,손승락,넥센,3.82,58,4,6,23,0,0.400,...,6,13,3,63,35,26,0,0,272,2015


In [29]:
# 8388, 1520 데이터 결합

df_tot = pd.concat([df_8388, df_1520], keys=['순위', '선수명', '팀명', 'ERA', 'G', 'CG', 'SHO', 'W', 'L', 'SV', 'HLD',
       'WPCT', 'TBF', 'IP', 'H', 'HR', 'BB', 'HBP', 'SO', 'R', 'ER', '년도'])

df_tot = df_tot.reset_index(drop=True)

In [30]:
df_tot.head(3)

,순위,선수명,팀명,ERA,G,CG,SHO,W,L,SV,...,TBF,IP,H,HR,BB,HBP,SO,R,ER,년도
0,1,최동원,롯데,2.89,38,16,1,9,16,4,...,863,208 2/3,202,17,51,8,148,89,67,1983
1,2,배경환,롯데,3.18,37,4,2,7,4,0,...,637,150,159,9,32,5,62,72,53,1983
2,3,천창호,롯데,3.42,27,0,0,7,3,0,...,349,81 2/3,91,12,12,6,38,39,31,1983


In [31]:
df_tot.tail(3)

,순위,선수명,팀명,ERA,G,CG,SHO,W,L,SV,...,TBF,IP,H,HR,BB,HBP,SO,R,ER,년도
1982,27,신재웅,SK,9.31,24,0,0,2,0,0,...,93,19 1/3,26,5,14,0,11,20,20,2020
1983,28,김찬호,SK,27.00,3,0,0,0,0,0,...,14,2,5,1,4,0,1,6,6,2020
1984,29,윤강민,SK,67.50,1,0,0,0,0,0,...,8,2/3,4,0,1,1,1,5,5,2020


## 연봉 데이터 추가

In [32]:
# 연봉 데이터 추가 (1)
# 열 추가

df_tot['Sal'] = np.nan
df_tot.head(3)

,순위,선수명,팀명,ERA,G,CG,SHO,W,L,SV,...,IP,H,HR,BB,HBP,SO,R,ER,년도,Sal
0,1,최동원,롯데,2.89,38,16,1,9,16,4,...,208 2/3,202,17,51,8,148,89,67,1983,NaN
1,2,배경환,롯데,3.18,37,4,2,7,4,0,...,150,159,9,32,5,62,72,53,1983,NaN
2,3,천창호,롯데,3.42,27,0,0,7,3,0,...,81 2/3,91,12,12,6,38,39,31,1983,NaN


In [33]:
# 연봉 데이터 추가 (2)
# 특정 사이트에서 연봉 정보를 찾아 이름에 맞게 데이터 삽입

URL = "http://www.statiz.co.kr/salary.php?opt=0&sopt=2015&te="
driver = webdriver.Chrome("../driver/chromedriver.exe")
driver.get(URL) # 스탯티즈 연봉 정보

html = driver.page_source
dom = BeautifulSoup(html, 'html.parser')

In [34]:
# 83~88 년도 연봉 데이터 없으므로, 15~20 년도 데이터만 추가

In [35]:
# 여러 시도를 위해 원본 데이터를 카피한 테스트용 데이터프레임 생성

df_test = df_tot.copy()

In [40]:
num = driver.find_element_by_css_selector('body > div > div.content-wrapper > div > section.content > div > div:nth-child(2) > div:nth-child(2) > div > div > div.box-body.no-padding > table > tbody')

num.text.split('\n')[1].split(' ')

['로저스', '2015', '한화', '170,000', '2.836']

In [41]:
# 사이트 버튼 클릭 및 정보 조회를 위한 여러 selenium 명령 / 변수 선언 / 연봉 정보 추출

sal_info = []

# 연도 클릭
for a in range (35, 41):
    driver.find_element_by_css_selector('body > div > div.content-wrapper > div > section.content > div > div:nth-child(2) > div:nth-child(1) > div > div:nth-child(1) > div > button').click()

    time.sleep(0.7)

    driver.find_element_by_css_selector(f'body > div > div.content-wrapper > div > section.content > div > div:nth-child(2) > div:nth-child(1) > div > div:nth-child(1) > div > ul > div > button:nth-child({a})').click()
  
    time.sleep(0.7)

    html = driver.page_source
    dom = BeautifulSoup(html, 'html.parser')

    team = dom.select('body > div > div.content-wrapper > div > section.content > div > div:nth-child(2) > div:nth-child(1) > div > div:nth-child(2) > div > ul > div > button')

    for b in range(2, len(team)+1):
        # 팀 클릭
        driver.find_element_by_css_selector('body > div > div.content-wrapper > div > section.content > div > div:nth-child(2) > div:nth-child(1) > div > div:nth-child(2) > div > button').click()

        time.sleep(0.7)

        driver.find_element_by_css_selector(f'body > div > div.content-wrapper > div > section.content > div > div:nth-child(2) > div:nth-child(1) > div > div:nth-child(2) > div > ul > div > button:nth-child({b})').click()

        time.sleep(0.7)

        num = driver.find_element_by_css_selector('body > div > div.content-wrapper > div > section.content > div > div:nth-child(2) > div:nth-child(2) > div > div > div.box-body.no-padding > table > tbody')

        # 연봉 테이블 추출
        sal_info.append(num.text.split('\n')[1:])


In [42]:
sal_test = sal_info.copy()
sal_test

[['윤석민 2015 KIA 125,000 3.873',
  '필 2015 KIA 90,000 3.057',
  '양현종 2015 KIA 75,000 8.135',
  '이범호 2015 KIA 65,000 2.778',
  '김주찬 2015 KIA 50,000 3.164',
  '나지완 2015 KIA 20,000 -0.144',
  '김원섭 2015 KIA 15,000 1.448',
  '김병현 2015 KIA 15,000 -0.193',
  '심동섭 2015 KIA 14,000 0.545',
  '최영필 2015 KIA 13,000 1.844',
  '임준혁 2015 KIA 13,000 2.492',
  '김민우 2015 KIA 12,000 0.619',
  '신종길 2015 KIA 12,000 0.888',
  '김진우 2015 KIA 12,000 0.494',
  '김광수 2015 KIA 10,000 0.549',
  '김태영 2015 KIA 8,800 0.273',
  '이홍구 2015 KIA 7,000 0.768',
  '한승혁 2015 KIA 7,000 0.376',
  '김다원 2015 KIA 7,000 0.354',
  '이성우 2015 KIA 6,500 -0.108',
  '한기주 2015 KIA 6,500 0.134',
  '유창식 2015 KIA 6,000',
  '백용환 2015 KIA 6,000 0.783',
  '김호령 2015 KIA 6,000 0.851',
  '강한울 2015 KIA 6,000 -0.945',
  '곽정철 2015 KIA 5,500',
  '김주형 2015 KIA 5,500 -0.831',
  '최용규 2015 KIA 5,000 -1.202',
  '박준표 2015 KIA 4,500 0.206',
  '홍건희 2015 KIA 4,500 -0.122',
  '신창호 2015 KIA 3,700 -0.032',
  '박찬호 2015 KIA 3,700 -1.174',
  '이호신 2015 KIA 3,600 -0.351'

In [43]:
sal_test[0][0].split(' ')
# 이름: sal_test[0][0].split(' ')[0]
# 연도: sal_test[0][0].split(' ')[1]
# 연봉: sal_test[0][0].split(' ')[3]

['윤석민', '2015', 'KIA', '125,000', '3.873']

In [44]:
# 연봉 테이블 정리

sal_tmp = []

for x in range(len(sal_test)):
    if len(sal_test[x]) > 0:
        for y in range(len(sal_test[x])):
            name = sal_test[x][y].split(' ')[0]
            year = sal_test[x][y].split(' ')[1]
            pay = sal_test[x][y].split(' ')[3]
            sal_tmp.append([name, year, pay])

In [45]:
# 정리된 연봉 정보 데이터 프레임에 기입

for x in range(len(sal_tmp)):
    name = sal_tmp[x][0]
    year = sal_tmp[x][1]
    pay = sal_tmp[x][2]
    df_test.loc[(df_test.선수명 == name) & (df_test.년도 == int(year)), 'Sal'] = pay

In [286]:
# 연봉정보 기입 여부 확인

df_test.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1985 entries, 0 to 1984
Data columns (total 23 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   순위      1985 non-null   object
 1   선수명     1985 non-null   object
 2   팀명      1985 non-null   object
 3   ERA     1985 non-null   object
 4   G       1985 non-null   object
 5   CG      1985 non-null   object
 6   SHO     1985 non-null   object
 7   W       1985 non-null   object
 8   L       1985 non-null   object
 9   SV      1985 non-null   object
 10  HLD     1985 non-null   object
 11  WPCT    1985 non-null   object
 12  TBF     1985 non-null   object
 13  IP      1985 non-null   object
 14  H       1985 non-null   object
 15  HR      1985 non-null   object
 16  BB      1985 non-null   object
 17  HBP     1985 non-null   object
 18  SO      1985 non-null   object
 19  R       1985 non-null   object
 20  ER      1985 non-null   object
 21  년도      1985 non-null   int64 
 22  Sal     1239 non-null   

In [47]:
# 타자의 연봉 정보도 들어가 있어서 실제 스탯티즈 연봉 정보가 모두 반영되는 것이 아님
# 현재 우리는 투수에 대해서만 확인 중
# 그로 인해 기입 된 정보 양이 적은 것임 (오류로 인함이 아님)

## 문제2: 1980년대 최동원이 활동하던 투수들의 연봉 및 데이터를 2010년대 투수들의 연봉 및 데이터로 맵핑 변환 문제 해결

* 해결 논리:
  - 당시 연봉 정보가 부족하므로 2010년대 투수들의 연봉을 이용하여 머신러닝, 검증하고
  - 이후 1980년대 투수들의 데이터를 사용하여 연봉이 어느 정도 될지 예측
  - 이를 위해 위에서 2010년대 정보 중 과거에는 집계하지 않았던 데이터는 삭제 하였음

<br>

* 이후 해결 과정:
  - 먼저 연봉 범주 생성 (eg. 2억~2억 9천: 2억 대, 3억~3억 9천: 3억 대...)
  - 2010년대 투수 중 연봉 정보가 있는 투수들의 데이터를 이용하여 머신러닝
  - 최동원 선수의 성적을 반영하여 얼마가 될지 확인

## 문제3: 최동원의 성적이라면 연봉이 어떨지 예측

In [422]:
# 연봉 정보만 빼서 새로운 데이터 프레임 선언 >> 머신러닝 이용 용이를 위해

df_sal = df_test[(df_test.Sal.notnull())]
df_sal.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1239 entries, 452 to 1984
Data columns (total 23 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   순위      1239 non-null   object
 1   선수명     1239 non-null   object
 2   팀명      1239 non-null   object
 3   ERA     1239 non-null   object
 4   G       1239 non-null   object
 5   CG      1239 non-null   object
 6   SHO     1239 non-null   object
 7   W       1239 non-null   object
 8   L       1239 non-null   object
 9   SV      1239 non-null   object
 10  HLD     1239 non-null   object
 11  WPCT    1239 non-null   object
 12  TBF     1239 non-null   object
 13  IP      1239 non-null   object
 14  H       1239 non-null   object
 15  HR      1239 non-null   object
 16  BB      1239 non-null   object
 17  HBP     1239 non-null   object
 18  SO      1239 non-null   object
 19  R       1239 non-null   object
 20  ER      1239 non-null   object
 21  년도      1239 non-null   int64 
 22  Sal     1239 non-null 

In [423]:
# 연봉 데이터 타입 변환을 위해 콤마 제거

df_sal['Sal'] = df_sal['Sal'].str.replace(',', '')
df_sal

C:\Users\User\AppData\Local\Temp\ipykernel_18688\4201383398.py:3: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



,순위,선수명,팀명,ERA,G,CG,SHO,W,L,SV,...,IP,H,HR,BB,HBP,SO,R,ER,년도,Sal
452,1,김정인,넥센,0.00,1,0,0,0,0,0,...,1,0,0,0,0,0,0,0,2015,3000
453,2,양훈,넥센,1.41,16,0,0,2,1,0,...,38 1/3,33,1,14,0,15,9,6,2015,15000
454,3,조상우,넥센,3.09,70,0,0,8,5,5,...,93 1/3,65,4,41,6,89,36,32,2015,17000
455,4,밴헤켄,넥센,3.62,32,0,0,15,8,0,...,196 2/3,190,14,67,1,193,92,79,2015,65000
456,5,손승락,넥센,3.82,58,0,0,4,6,23,...,61 1/3,73,6,13,3,63,35,26,2015,70000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1980,25,김택형,SK,8.61,31,0,0,1,3,0,...,23,26,5,20,0,22,26,22,2020,3700
1981,26,양선률,SK,9.00,1,0,0,0,0,0,...,1,1,1,2,0,0,1,1,2020,2700
1982,27,신재웅,SK,9.31,24,0,0,2,0,0,...,19 1/3,26,5,14,0,11,20,20,2020,11000
1983,28,김찬호,SK,27.00,3,0,0,0,0,0,...,2,5,1,4,0,1,6,6,2020,2900


In [424]:
# 연봉 데이터 정수로 변환
df_sal['Sal'] = df_sal['Sal'].astype(int)
df_sal = df_sal.reset_index(drop=True)
df_sal

C:\Users\User\AppData\Local\Temp\ipykernel_18688\2950672990.py:2: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



,순위,선수명,팀명,ERA,G,CG,SHO,W,L,SV,...,IP,H,HR,BB,HBP,SO,R,ER,년도,Sal
0,1,김정인,넥센,0.00,1,0,0,0,0,0,...,1,0,0,0,0,0,0,0,2015,3000
1,2,양훈,넥센,1.41,16,0,0,2,1,0,...,38 1/3,33,1,14,0,15,9,6,2015,15000
2,3,조상우,넥센,3.09,70,0,0,8,5,5,...,93 1/3,65,4,41,6,89,36,32,2015,17000
3,4,밴헤켄,넥센,3.62,32,0,0,15,8,0,...,196 2/3,190,14,67,1,193,92,79,2015,65000
4,5,손승락,넥센,3.82,58,0,0,4,6,23,...,61 1/3,73,6,13,3,63,35,26,2015,70000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1234,25,김택형,SK,8.61,31,0,0,1,3,0,...,23,26,5,20,0,22,26,22,2020,3700
1235,26,양선률,SK,9.00,1,0,0,0,0,0,...,1,1,1,2,0,0,1,1,2020,2700
1236,27,신재웅,SK,9.31,24,0,0,2,0,0,...,19 1/3,26,5,14,0,11,20,20,2020,11000
1237,28,김찬호,SK,27.00,3,0,0,0,0,0,...,2,5,1,4,0,1,6,6,2020,2900


In [425]:
df_sal.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1239 entries, 0 to 1238
Data columns (total 23 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   순위      1239 non-null   object
 1   선수명     1239 non-null   object
 2   팀명      1239 non-null   object
 3   ERA     1239 non-null   object
 4   G       1239 non-null   object
 5   CG      1239 non-null   object
 6   SHO     1239 non-null   object
 7   W       1239 non-null   object
 8   L       1239 non-null   object
 9   SV      1239 non-null   object
 10  HLD     1239 non-null   object
 11  WPCT    1239 non-null   object
 12  TBF     1239 non-null   object
 13  IP      1239 non-null   object
 14  H       1239 non-null   object
 15  HR      1239 non-null   object
 16  BB      1239 non-null   object
 17  HBP     1239 non-null   object
 18  SO      1239 non-null   object
 19  R       1239 non-null   object
 20  ER      1239 non-null   object
 21  년도      1239 non-null   int64 
 22  Sal     1239 non-null   

In [426]:
# 연봉 범주 축소 전 조회

df_sal.Sal.unique()

array([  3000,  15000,  17000,  65000,  70000,   7900,   4900,  45000,
        30000,   3800,  20000,  19000,  11000,   3300,   3400,   5500,
         4400,   2700,   8800,      0,   6000,   2900,   6200,  40000,
        10000,  11500, 100000,   6800, 120000,   9000,   3200,   7300,
         2800,   6300,  63000,  12000,   9500,   3600,   6500,  25000,
        22000,   5000,   4600,  18000,  50000,   4500,  75000,  80000,
        14000,   4000,   7000,   3700, 170000,   8500,   5200,   5100,
        28000,  16000,  55000,  13000,   8900,   7200,   4700,   3900,
        12500,  60000,   7500,   3500,   9200,   7700,  17400,  13500,
         6600,  35000,  85000,  16500,  52500,   8000,   8400,  10500,
        14500,   3100,   4800,   2400,   4200, 210000,   5400,  21000,
        33000,   8300,   8100, 150000,  27000,   5300,   5700,  22500,
         8700,   6400,  23000, 135000,  23500, 111000,  15500,   7100,
        24000, 230000,  29000,  31000,   7600,   5800,  18800,  18500,
      

In [427]:
# 연봉 범주 축소 (카테고리화)
# 천만 단위 연봉 >> 0 : 1억 미만
# 1~5억 >> 1
# 5억~10억 >> 2
# 10억 이상 >> 3

for x, y in df_sal.iterrows():
    if df_sal.loc[x, 'Sal'] < 10000:
        df_sal.loc[x, 'Sal_cl'] = 0
    elif df_sal.loc[x, 'Sal'] > 10000 and df_sal.loc[x, 'Sal'] < 50000:
        df_sal.loc[x, 'Sal_cl'] = 1
    elif df_sal.loc[x, 'Sal'] > 50000 and df_sal.loc[x, 'Sal'] < 100000:
        df_sal.loc[x, 'Sal_cl'] = 2
    else:
        df_sal.loc[x, 'Sal_cl'] = 3

df_sal.head()

,순위,선수명,팀명,ERA,G,CG,SHO,W,L,SV,...,H,HR,BB,HBP,SO,R,ER,년도,Sal,Sal_cl
0,1,김정인,넥센,0.00,1,0,0,0,0,0,...,0,0,0,0,0,0,0,2015,3000,0.0
1,2,양훈,넥센,1.41,16,0,0,2,1,0,...,33,1,14,0,15,9,6,2015,15000,1.0
2,3,조상우,넥센,3.09,70,0,0,8,5,5,...,65,4,41,6,89,36,32,2015,17000,1.0
3,4,밴헤켄,넥센,3.62,32,0,0,15,8,0,...,190,14,67,1,193,92,79,2015,65000,2.0
4,5,손승락,넥센,3.82,58,0,0,4,6,23,...,73,6,13,3,63,35,26,2015,70000,2.0


## 머신 러닝 및 연봉 예측 진행

In [428]:
df_sal.columns

Index(['순위', '선수명', '팀명', 'ERA', 'G', 'CG', 'SHO', 'W', 'L', 'SV', 'HLD',
       'WPCT', 'TBF', 'IP', 'H', 'HR', 'BB', 'HBP', 'SO', 'R', 'ER', '년도',
       'Sal', 'Sal_cl'],
      dtype='object')

In [429]:
# 대시 부호 / 빈칸은 오류를 일으키므로 0으로 임의 변경
df_sal = df_sal.replace('-', '0')
df_sal = df_sal.replace('', '0')
df_sal.head(3)

,순위,선수명,팀명,ERA,G,CG,SHO,W,L,SV,...,H,HR,BB,HBP,SO,R,ER,년도,Sal,Sal_cl
0,1,김정인,넥센,0.00,1,0,0,0,0,0,...,0,0,0,0,0,0,0,2015,3000,0.0
1,2,양훈,넥센,1.41,16,0,0,2,1,0,...,33,1,14,0,15,9,6,2015,15000,1.0
2,3,조상우,넥센,3.09,70,0,0,8,5,5,...,65,4,41,6,89,36,32,2015,17000,1.0


In [430]:
# IP의 /부호도 오류를 일으키므로 제거
df_sal.IP = df_sal.IP.str.replace('1/3', '')
df_sal.IP = df_sal.IP.str.replace('2/3', '')
df_sal = df_sal.replace('', '0')
df_sal.head(3)

,순위,선수명,팀명,ERA,G,CG,SHO,W,L,SV,...,H,HR,BB,HBP,SO,R,ER,년도,Sal,Sal_cl
0,1,김정인,넥센,0.00,1,0,0,0,0,0,...,0,0,0,0,0,0,0,2015,3000,0.0
1,2,양훈,넥센,1.41,16,0,0,2,1,0,...,33,1,14,0,15,9,6,2015,15000,1.0
2,3,조상우,넥센,3.09,70,0,0,8,5,5,...,65,4,41,6,89,36,32,2015,17000,1.0


In [431]:
# 데이터와 답 분리 / 훈련, 검증 데이터 분리

X = df_sal.drop(['순위', '팀명', '선수명', 'Sal', 'Sal_cl'], axis = 1)
y = df_sal['Sal_cl']

from sklearn.model_selection import train_test_split
import numpy as np

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2)

In [432]:
# 훈련/검증 데이터 분리 정도 확인

import plotly.graph_objects as go

fig = go.Figure()
fig.add_trace(go.Histogram(x=X_train['년도'], name = 'Train'))
fig.add_trace(go.Histogram(x=X_test['년도'], name = 'Test'))

fig.update_layout(barmode='overlay')
fig.update_traces(opacity=0.75)
fig.show()

In [433]:
# Decision Tree training

from sklearn.model_selection import GridSearchCV
from sklearn.metrics import accuracy_score
from sklearn.tree import DecisionTreeClassifier

sal_tree = DecisionTreeClassifier(max_depth=3)
sal_tree.fit(X_train, y_train)

y_pred_tr = sal_tree.predict(X_train)
y_pred_test = sal_tree.predict(X_test)

print('Train Accuray : ', accuracy_score(y_train, y_pred_tr))
print('Test Accuray : ', accuracy_score(y_test, y_pred_test))

Train Accuray :  0.7245206861755802
Test Accuray :  0.7338709677419355


In [434]:
dict(zip(X_train.columns, sal_tree.feature_importances_))

{'ERA': 0.02388246923090856,
 'G': 0.0,
 'CG': 0.0,
 'SHO': 0.0,
 'W': 0.5062461929599581,
 'L': 0.0,
 'SV': 0.21665873446745,
 'HLD': 0.053217618436215974,
 'WPCT': 0.0,
 'TBF': 0.0,
 'IP': 0.16352747498752884,
 'H': 0.0,
 'HR': 0.03646750991793857,
 'BB': 0.0,
 'HBP': 0.0,
 'SO': 0.0,
 'R': 0.0,
 'ER': 0.0,
 '년도': 0.0}

In [435]:
# GridSearch CV 활용, 최적화된 max_depth 추적

from sklearn.model_selection import GridSearchCV
from sklearn.tree import DecisionTreeClassifier

params = {'max_depth' : [2, 3, 4, 5, 7, 10, 15, 20]}

sal_tree = DecisionTreeClassifier(max_depth=2)
gridsearch = GridSearchCV(estimator=sal_tree, param_grid = params, cv=5)
gridsearch.fit(X, y)

# GridSearch CV 결과

import pprint

pp = pprint.PrettyPrinter(indent=4)
pp.pprint(gridsearch.cv_results_)

{   'mean_fit_time': array([0.00700626, 0.0084075 , 0.00680618, 0.00760679, 0.00840769,
       0.01020927, 0.01121006, 0.01161056]),
    'mean_score_time': array([0.00260253, 0.0028028 , 0.002002  , 0.00200191, 0.00200176,
       0.00200186, 0.00200195, 0.00240235]),
    'mean_test_score': array([0.68764529, 0.68682905, 0.62611336, 0.63416482, 0.59213465,
       0.56389905, 0.53809912, 0.53728614]),
    'param_max_depth': masked_array(data=[2, 3, 4, 5, 7, 10, 15, 20],
             mask=[False, False, False, False, False, False, False, False],
       fill_value='?',
            dtype=object),
    'params': [   {'max_depth': 2},
                  {'max_depth': 3},
                  {'max_depth': 4},
                  {'max_depth': 5},
                  {'max_depth': 7},
                  {'max_depth': 10},
                  {'max_depth': 15},
                  {'max_depth': 20}],
    'rank_test_score': array([1, 2, 4, 3, 5, 6, 7, 8]),
    'split0_test_score': array([0.69354839, 0.7096774

In [436]:
# GridSearch를 이용한 최적 값 확인

gridsearch.best_estimator_, gridsearch.best_score_, gridsearch.best_params_

(DecisionTreeClassifier(max_depth=2), 0.6876452918897741, {'max_depth': 2})

## 최동원 선수 데이터 기입하여 연봉 예측

In [443]:
# 최동원 선수 데이터 확인

df_8388[(df_8388['선수명']=='최동원')]

,순위,선수명,팀명,ERA,G,CG,SHO,W,L,SV,...,TBF,IP,H,HR,BB,HBP,SO,R,ER,년도
0,1,최동원,롯데,2.89,38,16,1,9,16,4,...,863,208,202,17,51,8,148,89,67,1983
57,3,최동원,롯데,2.40,51,14,1,27,13,6,...,1132,284,228,18,68,14,223,91,76,1984
114,1,최동원,롯데,1.92,42,14,4,20,9,8,...,865,225,170,7,41,8,161,60,48,1985
183,1,최동원,롯데,1.55,39,17,4,19,14,2,...,1039,267,204,7,55,6,208,60,46,1986
274,3,최동원,롯데,2.81,32,15,4,14,12,2,...,920,224,218,6,61,7,163,80,70,1987
360,1,최동원,롯데,2.05,16,3,1,7,3,3,...,349,83,77,4,24,1,83,24,19,1988


In [442]:
# 오류 발생 부분 변경

df_8388 = df_8388.replace('-', '0')
df_8388 = df_8388.replace('', '0')
df_8388.IP = df_8388.IP.str.replace('1/3', '')
df_8388.IP = df_8388.IP.str.replace('2/3', '')
df_8388 = df_8388.replace('', '0')

In [449]:
choi = df_8388[(df_8388['선수명']=='최동원')].drop(['순위', '팀명', '선수명'], axis = 1)

In [457]:
X = df_sal.drop(['순위', '팀명', '선수명', 'Sal', 'Sal_cl'], axis = 1)
y = df_sal['Sal_cl']

from sklearn.model_selection import train_test_split
import numpy as np

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2)

sal_tree = DecisionTreeClassifier(max_depth=2)
sal_tree.fit(X_train, y_train)

y_pred_tr = sal_tree.predict(X_train)
y_pred_test = sal_tree.predict(X_test)

print('Train Accuray : ', accuracy_score(y_train, y_pred_tr))
print('Test Accuray : ', accuracy_score(y_test, y_pred_test))

Train Accuray :  0.7033299697275479
Test Accuray :  0.7016129032258065


In [458]:
print('최동원 선수 연봉: ', sal_tree.predict_proba(choi))

최동원 선수 연봉:  [[0.1372549  0.17647059 0.31372549 0.37254902]
 [0.1372549  0.17647059 0.31372549 0.37254902]
 [0.1372549  0.17647059 0.31372549 0.37254902]
 [0.1372549  0.17647059 0.31372549 0.37254902]
 [0.1372549  0.17647059 0.31372549 0.37254902]
 [0.46712803 0.36678201 0.10034602 0.06574394]]


In [462]:
choi_sal = pd.DataFrame(sal_tree.predict_proba(choi))
choi_sal.columns = ['1억 미만','1억 이상, 5억 미만', '5억 이상 10억 미만', '10억 이상']
choi_sal

,1억 미만,"1억 이상, 5억 미만",5억 이상 10억 미만,10억 이상
0,0.137255,0.176471,0.313725,0.372549
1,0.137255,0.176471,0.313725,0.372549
2,0.137255,0.176471,0.313725,0.372549
3,0.137255,0.176471,0.313725,0.372549
4,0.137255,0.176471,0.313725,0.372549
5,0.467128,0.366782,0.100346,0.065744


## 결론

1. 83년~87년까지 최동원 선수는 2010년대 선수들의 성적과 비교하여 5억 / 10억 이상의 연봉을 받을 수 있을 정도의 결과를 냈다고 예상할 수 있음

<br>

2. 88년도에는 비교적 성적이 부진하여 5억 미만의 연봉을 받을 정도로 보임

<br>

3. 88년도 결과를 고려하더라도 5년간의 성적이 뛰어나므로 종합적으로 최동원 선수가 2010년대에 활동했더라면 충분히 10억 이상의 연봉을 받았을 것이라고 예측됨

<br>

* 물론, 데이터에 나오지 않은 다양한 요소들이 평가되어 연봉이 책정 되었을 것임. 현재 사용한 데이터는 결과를 보여주는 것이고, 해당 선수가 앞으로 시합에 어떤 결과를 가져올지 등을 고려하여 책정 될 수 있음. 이러한 부분에 대해 어떻게 예상할 것인지, 지표를 어떻게 만들 것인지 등 생각해 볼 것이 많아보임.

#### 배운 점

* 처음에는 연봉의 분류를 천만단위, 억 단위로 쪼개었기에 대상 수가 많아서 예측의 결과가 30%를 웃돌았음.
* 이를 해결하기 위해 좀 더 넓은 범위의 연봉을 한 카테고리에 담았음 >> 예측률이 70% 정도로 올랐고, 과적합의 모습은 보이지 않았음
* 결과가 rough 해지더라도, 데이터의 수가 적을 경우에는 머신러닝 답안의 카테고리 수를 최대한 적게 만들어 정확도를 높이는 방법을 쓰는 것도 좋은 것 같아보임.